In [4]:
!pip3 install transformers torch torchvision torchaudio tensorflow
from transformers import pipeline, set_seed, GPT2Tokenizer
import os
import nltk

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 52.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.1/737.1 kB 25.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
descript-audiotools-unofficial 0.7.4 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 6.32.1 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3.12 -m pip install --upgrade pip


# Experiment 1: Text Generation

In [5]:
set_seed(45)

In [6]:
prompt = "The future of Artificial Intelligence is"

### BERT

In [7]:
fast_generator = pipeline('text-generation', model="bert-base-uncased")
output_fast = fast_generator(prompt, max_length=25, num_return_sequences=1)
print(output_fast[0]['generated_text'])

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The future of Artificial Intelligence is.................


### RoBERTa

In [8]:
fast_generator = pipeline('text-generation', model="roberta-base")
output_fast = fast_generator(prompt, max_length=25, num_return_sequences=1)
print(output_fast[0]['generated_text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The future of Artificial Intelligence is


### BART

In [9]:
fast_generator = pipeline('text-generation', model="facebook/bart-base")
output_fast = fast_generator(prompt, max_length=25, num_return_sequences=1)
print(output_fast[0]['generated_text'])

Some weights of BartForCausalLM were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The future of Artificial Intelligence isokes sectionsippedippedbankbankbankokesokes �bankbank autopsy autopsybankbank


# Experiment 2: Masked Language Modeling (Missing Word)

### BERT

In [29]:
mask_filler = pipeline("fill-mask", model="bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [30]:
masked_sentence = "The goal of Generative AI is to [MASK] new content."
preds = mask_filler(masked_sentence)

for p in preds:
    print(f"{p['token_str']}: {p['score']:.2f}")

create: 0.54
generate: 0.16
produce: 0.05
develop: 0.04
add: 0.02


### RoBERTa

In [31]:
mask_filler = pipeline("fill-mask", model="roberta-base")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [32]:
masked_sentence = "The goal of Generative AI is to <mask> new content."
preds = mask_filler(masked_sentence)

for p in preds:
    print(f"{p['token_str']}: {p['score']:.2f}")

 create: 0.37
 generate: 0.37
 discover: 0.08
 find: 0.02
 provide: 0.02


### BART

In [33]:
mask_filler = pipeline("fill-mask", model="facebook/bart-base")  

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [34]:
masked_sentence = "The goal of Generative AI is to <mask> new content."
preds = mask_filler(masked_sentence)

for p in preds:
    print(f"{p['token_str']}: {p['score']:.2f}")

 create: 0.07
 help: 0.07
 provide: 0.06
 enable: 0.04
 improve: 0.03


# Experiment 3: Question Answering

In [35]:
question = "What are the risks?"
context = "Generative AI poses significant risks such as hallucinations, bias, and deepfakes."

### BERT

In [36]:
qa_pipeline = pipeline("question-answering", model="bert-base-uncased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [37]:
res = qa_pipeline(question=question, context=context)
print(f"Q: {question}")
print(f"A: {res['answer']}")

Q: What are the risks?
A: hallucinations


### RoBERTa

In [38]:
qa_pipeline = pipeline("question-answering", model="roberta-base")

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [39]:
res = qa_pipeline(question=question, context=context)
print(f"Q: {question}")
print(f"A: {res['answer']}")

Q: What are the risks?
A: Generative AI poses


### BART

In [40]:
qa_pipeline = pipeline("question-answering", model="facebook/bart-base")

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [41]:
res = qa_pipeline(question=question, context=context)
print(f"Q: {question}")
print(f"A: {res['answer']}")

Q: What are the risks?
A: Generative AI poses significant risks


# Observation Table

| Task               | Model     | Classification (Success / Failure) | Observation (What actually happened?)                                                                 | Why did this happen? (Architectural Reason) |
|--------------------|-----------|------------------------------------|----------------------------------------------------------------------------------------------------------|---------------------------------------------|
| Text Generation    | BERT      | Failure                            | Generated only dots (`.................`) instead of a meaningful continuation                           | BERT is an encoder-only model trained using Masked Language Modeling (MLM). It does not learn next-token prediction, which is required for autoregressive text generation. |
|                    | RoBERTa   | Failure                            | Returned only the original prompt without any continuation                                                | Like BERT, RoBERTa is encoder-only and trained with MLM. It lacks a causal decoder, so it cannot generate new tokens sequentially. |
|                    | BART      | Failure                            | Produced incoherent, repetitive, and nonsensical text (`bankbank autopsy autopsy...`)                   | BART is an encoder–decoder model trained for denoising, not causal generation. Using it with a decoder-only text-generation pipeline causes unstable output. |
| Fill-Mask (MLM)    | BERT      | Success                            | Correctly predicted relevant verbs with high confidence: `create (0.54)`, `generate (0.16)`             | BERT is explicitly trained for masked token prediction, making it well-suited for MLM tasks. |
|                    | RoBERTa   | Success                            | Produced strong and semantically correct predictions: `create (0.37)`, `generate (0.37)`               | RoBERTa improves MLM training with more data, dynamic masking, and removal of NSP, resulting in better contextual understanding. |
|                    | BART      | Partial Success                    | Returned reasonable but low-confidence predictions: `create (0.07)`, `help (0.07)`, `provide (0.06)`   | BART uses span corruption and denoising objectives rather than single-token MLM, so it is less precise for fill-mask tasks. |
| Question Answering | BERT      | Partial Success                    | Returned only one correct risk: `hallucinations`                                                         | Base BERT is not fine-tuned for extractive QA. Without task-specific fine-tuning, it often predicts incomplete answer spans. |
|                    | RoBERTa   | Failure                            | Returned an incorrect span: `Generative AI poses`                                                        | Although RoBERTa has strong representations, it requires QA fine-tuning to accurately predict start–end answer spans. |
|                    | BART      | Partial Success                    | Returned an overlong answer: `Generative AI poses significant risks`                                     | BART-base is not optimized for extractive QA. Without fine-tuning, it tends to select broader spans instead of concise answers. |
